In [ ]:
from __future__ import print_function

from intervaltree import IntervalTree, Interval
from collections import defaultdict
import sys

In [ ]:
def overlap_intervals(self, other):
    splits = (self | other)
    splits.split_overlaps()
    self_int_other = IntervalTree(filter(lambda r: self.overlaps(r) and other.overlaps(r), splits))
    return self_int_other


def overlap_all(trees):
    if len(trees) in (0, 1):
        return None

    overlap = overlap_intervals(trees[0], trees[1])
    for t in trees[2:]:
        overlap = overlap_intervals(overlap, t)
    overlap.merge_overlaps(data_reducer)

    return overlap


def data_reducer(current_data, new_data):
    return current_data | new_data

In [ ]:
gtrees = defaultdict(dict)
all_genes = set()
files = defaultdict(lambda: defaultdict(list))

In [ ]:
for f in ('vqc', 'iqc', 'sqc', 'mqc'):
    for line in open(f): 
            x = line.rstrip().split(' ')
            if x[2] != 'Extensive':
                files[f][x[14]].append((int(x[3]), int(x[4])))

In [ ]:
gtrees = defaultdict(dict)
all_genes = set()
for f in files:
    for g in files[f]:
        all_genes.add(g)
        gtrees[g][f] = IntervalTree(Interval(iv[0], iv[1], set([f]))
                                    for iv in files[f][g])

In [ ]:
overlap_trees = {}
for gene in all_genes:
    overlap_trees[gene] = overlap_all(list(gtrees[gene].values()))
    if overlap_trees[gene]:
        print('@', gene, end=": \n ")
        print(overlap_trees[gene]) 

In [ ]:
for i in overlap_trees['Desulfovibrio_vulgaris_DP4']:
   print(i)

In [ ]:
#overlap_trees['Desulfovibrio_vulgaris_DP4'].remove_envelop(0, 179544)
#overlap_trees['Desulfovibrio_vulgaris_DP4'].remove_overlap(0, 179544)   

print(overlap_trees['Desulfovibrio_vulgaris_DP4'].begin())
print(overlap_trees['Desulfovibrio_vulgaris_DP4'].end())
print ('done')

In [ ]:
#out=open('case', 'a')
#for i in overlap_trees['Enterococcus_faecalis_V583']:
   #       print(i,file=out)


In [ ]:
cutoff =10
out =open('case','w+')
overlap_trees = {}
for gene in all_genes:
    overlap_trees[gene] = overlap_all(list(gtrees[gene].values()))
    if overlap_trees[gene]:
        #print('@',gene,":",overlap_trees[gene].begin( ),",",overlap_trees[gene].end(), len(overlap_trees[gene])) 
        if len(overlap_trees[gene]) >cutoff:
                for i in overlap_trees[gene]:
                        print(gene,',',i,',',file=out)
                    
print ('done')                


In [ ]:
 #parse case files
gdict=defaultdict(list)
input=open('case','r')
for line in input:
        line=line.split(',')
        #print(line)
        gene=line[0]
        x=line[1].split('(')
        x=x[1]
        y=line[2]
        #print(x,"-",y)
        gdict[line[0]].append(x)
        gdict[line[0]].append(y)


for g in gdict.keys(): 
         i=1
         l=len(gdict[g])
         min =20700000
         max =-1
         sum=0
         while(i <l-1) :
                gap=abs( int(gdict[g][i+1]) -int(gdict[g][i]) )
                sum+=gap
                if min >gap: 
                        min=gap
                if max <gap:
                        max=gap
                i=i+2
         print(g,"has ",l/2,"misassemblies with min/max/average gaps:",str(min), "/",str(max),"/",(sum/l))


         